# Chains

* [1. Chains and Why They Are Used](#chains)
    * [1.1. LLMChain](#LLMChain) 
    * [1.2. Parsers](#parsers) 
    * [1.3. Conversational Chain (Memory)](#memory) 
    * [1.4. Sequential Chain](#seq)
    * [1.5. Debug](#debug) 
    * [1.6. Custom](#custom) 
* [2. YouTube Video Summarizer](#video_summarizer)
* [3. Creating Voice Assistant](#voice_assistant)
* [4. Code Comprehension - Twitter Algorithm](#code_understanding)
* [5. Recommendation Engine for Songs](#recommendation)
* [6. Self-Critique Chain](#critique)
* [7. Additional Resources](#resources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="chains">
    
## 1. Chains and the Reasons to Use them
    
</a>

Chains are responsible for creating an end-to-end pipeline to use LLMs. Chains join the model, prompt, memory, parsing output, debugging capability. They also provide an easy-to-use interface. 

A chain will:
1) receive the user’s query as an input;
2) process the LLM’s response;
3) return the output to the user.

It is possible to design a custom pipeline by inheriting the `Chain` class. 

<hr>
<a class="anchor" id="LLMChain">
    
### 1.1. LLMChain
    
</a>

There are several methods to use chains:
- **`__ call __`** pass an input directly to the object while initializing it; will return the input variable and the model’s response under the text key;
- **`.apply()`** pass multiple inputs at once and receive a list for each input;
- **`.generate()`** returns an instance of `LLMResult`, which provides more information;
- **`.predict()`** pass multiple (or single) inputs for a single prompt;
- **`.run()`** the same as .predict (they can be used interchangeably).

In [2]:
from langchain import PromptTemplate, OpenAI, LLMChain

In [3]:
prompt_template = "What is a word to replace the following: {word}?"


llm = OpenAI(model_name="text-davinci-003", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [4]:
# Passing input directly to object while initializing it >> __call__ 
llm_chain("artificial") 

{'word': 'artificial', 'text': '\n\nSynthetic'}

In [5]:
# Passing multiple inputs  >> .apply() 
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

[{'text': '\n\nSynthetic'}, {'text': '\n\nWisdom'}, {'text': '\n\nAutomaton'}]

In [6]:
# Return an instance of LLMResult with more information >> .generate()
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSynthetic', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nWisdom', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nAutomaton', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 46, 'prompt_tokens': 33, 'completion_tokens': 13}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('12354ad4-428d-4577-afed-9bf2f543fdd4')), RunInfo(run_id=UUID('846124ec-28fe-42f9-9bf3-818e7f79afa7')), RunInfo(run_id=UUID('fe91d0d7-e93e-41b0-a123-b1ba6bbe585e'))])

In [7]:
# Pass both the word and the context
prompt_template = "Looking at the context of '{context}'. What is an appropriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="present", context="object")

'\n\ngift'

In [8]:
# alternative to .predict()
llm_chain.run(word="present", context="object")

'\n\ngift'

In [9]:
llm_chain.predict(word="present", context="time") # or llm_chain.run(word="present", context="time")

'\n\nNow'

**Note**: To format the output we can use either parsers (see example below and refer to [notebook section](06.%20Prompting.ipynb#outputs)) or we can directly pass a prompt as a string to a Chain and initialize it using the .from_string() function as follows:
`LLMChain.from_string(llm=llm, template=template)`

<hr>
<a class="anchor" id="parsers">
    
### 1.2. Parsers
    
</a>

In [13]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, output_parser=output_parser, input_variables=[]),
    output_parser=output_parser)

llm_chain.predict()

['Synthetic',
 'Manufactured',
 'Imitation',
 'Fabricated',
 'Fake',
 'Mechanical',
 'Computerized',
 'Automated',
 'Simulated',
 'Artificial Intelligence',
 'Constructed',
 'Programmed',
 'Processed',
 'Algorithmic.']

<hr>
<a class="anchor" id="memory">
    
### 1.3. Conversational Chain (Memory)
    
</a>

LangChain provides a `ConversationalChain` to track previous prompts and responses using the `ConversationalBufferMemory` class.

In [16]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'great' as comma separated.")

' Excellent, superb, wonderful, terrific, outstanding, remarkable, splendid, grand, fabulous, magnificent, glorious, sublime.'

In [17]:
conversation.predict(input="And the next 4?")

' Amazing, remarkable, incredible, and phenomenal.'

<hr>
<a class="anchor" id="seq">
    
### 1.4. Sequential Chain
    
</a>

Sequantial chain is desined to concatenate multiple chains into one. For example, the `SimpleSequentialChain` instance created below will start running each chain from the first index and pass its response to the next one in the list:

In [18]:
from langchain.chains import SimpleSequentialChain

#overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two])

<hr>
<a class="anchor" id="debug">
    
### 1.5. Debug
    
</a>

To trace the inner work of a chain, one should set the `verbose` argument to `True` (if so, the output will depend on a specific application of chain).

In [19]:
template = """List all possible words as substitute for 'neural' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'neural' as comma separated.

Current conversation:




> Finished chain.


'Brainy, Nervous, Nerve-wracking, Synaptic, Cognitive, Cerebral, Mental, Intellective, Thoughtful, Mindful, Psychogenic, Psychical, Psychosomatic.'

<hr>
<a class="anchor" id="custom">
    
### 1.6. Custom Chain
    
</a>


It is possible to define your own chain for any custom task:
1. Define a class that inherits most of its functionalities from the `Chain` class;
2. Declare three methods: `input_keys`, `output_keys` and `_call` (declation will depend on the specifics of the task).

In [20]:
# Create a custom chain (ConcatenateChain) that returns a word's meaning and then suggests a replacement
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [21]:
# Declare each chain individually using the "LLMChain" class

# 1
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# 2
prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

In [24]:
# Call ConcatenateChain to merge the results of the chain_1 and chain_2
concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("intelligence")

print(f"Concatenated output:\n{concat_output}")

Concatenated output:


Intelligence is the ability to acquire and apply knowledge and skills. It is the capacity to think, reason, understand, and learn. It is also the ability to solve problems and adapt to new situations.

Wisdom


<hr>
<a class="anchor" id="video_summarizer">
    
## 2. YouTube Video Summarizer
    
</a>

It is possible build a tool to effectively extract key takeaways from YouTube videos. It can be done by leveraging Whisper to transcribe YouTube audio files and then create summarized output by using LangChain's summarization techniques (including stuff, refine, and map_reduce).

The **stuff** approach is the simplest and most naive one: all the text from the documents is used in a single prompt. This method may raise exceptions if all text is longer than the available context size of the LLM. The **map-reduce** and **refine** approaches offer more sophisticated ways to process and extract information from longer documents. The "map-reduce" method can be parallelized,so it is faster. The "refine" approach is sequential in nature, making it slower compared to the "map-reduce" method, but producing better results. The most suitable approach should be selected by considering the trade-offs between speed and quality.

**Whisper** is a cutting-edge, automatic speech recognition system developed by OpenAI. It has been trained on an impressive 680,000 hours of multilingual and multitasking supervised data sourced from the web.

STEPS:
- Download the desired YouTube audio file;
- Transcribe the audio with the help of Whisper;
- Summarize the transcribed text using LangChain (stuff, refine, and map_reduce);
- Adding multiple URLs to DeepLake database, and retrieving information. 

In [25]:
# SETUP

# !pip install langchain==0.0.208 deeplake openai tiktoken
# !pip install -q yt_dlp
# !pip install -q git+https://github.com/openai/whisper.git

######################################

# MacOS (requires https://brew.sh/)
#brew install ffmpeg

# Ubuntu
#sudo apt install ffmpeg

In [1]:
import os
from keys import OPENAI_API_KEY, ACTIVELOOP_TOKEN

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN

In [2]:
import yt_dlp

# Define function to download video from YouTube to a local file
def download_mp4_from_youtube(url, filename):
    # Set the options for the download
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }
    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

        
url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
filename = 'data/lecuninterview.mp4'

download_mp4_from_youtube(url, filename)

The whisper package that we installed earlier provides the `.load_model()` method to download the model and transcribe a video file. Multiple different models are available: `tiny`, `base`, `small`, `medium`, and `large` (each of them has tradeoffs between accuracy and speed). 

In [3]:
import whisper

model = whisper.load_model("base")

filename = 'data/lecuninterview.mp4'
result = model.transcribe(filename)

# To print out the obtained transcription
# print(result['text'])

/Users/iryna/Documents/projects/langchain_snippets/langenv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


**Note**: If an eeror about SSL certificate is raised while running the code above, have a look at the solution [here](https://stackoverflow.com/questions/68275857/urllib-error-urlerror-urlopen-error-ssl-certificate-verify-failed-certifica).

In [4]:
# Save result to a text file
with open ('output/text.txt', 'w') as file:  
    file.write(result['text'])

In [9]:
# Print out a chunk of the result
print(result['text'][:600])

 Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. I wanted to talk to


In [10]:
# Loading utilities from the LangChain library necessary to perform Summarization Step
from langchain import OpenAI, LLMChain  # to handle large texts
from langchain.chains.mapreduce import MapReduceChain  # to optimize
from langchain.prompts import PromptTemplate   # to construct prompt
from langchain.chains.summarize import load_summarize_chain  # to run summarization

llm = OpenAI(model_name="text-davinci-003", temperature=0)  # initialize an instance of OpenAI LLM

In [13]:
# Split input text into smaller chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

with open('output/text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:5]]  # only the 5 first chunks will be used

In [14]:
from langchain.chains.summarize import load_summarize_chain
import textwrap   

chain = load_summarize_chain(llm, chain_type="map_reduce")
output_summary = chain.run(docs)

# Format and print the output
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 Jan Le Ka is a professor at New York University and Chief AI Scientist at Fair, a fundamental AI
research lab. His research focuses on self-supervised learning, which has revolutionized natural
language processing. His latest paper is the Joint Embedding Predictive Architecture, which relates
to what is lacking in large language models. Self-supervised learning is a technique used to train
large neural networks to predict missing words in a piece of text. Generative models are used to
predict missing words in a text, but it is difficult to represent uncertain predictions. Attempts to
transfer self-supervised learning methods from language processing to images have not been
successful, but this approach has been successful in audio.


In [15]:
# To see the prompt template that is used with the map_reduce technique
print( chain.llm_chain.prompt.template )

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [16]:
# Experimenting with the prompt
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

In [17]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


- Jan LeCoon is a seminal figure in deep learning development and a long time proponent of self-supervised learning
- Discussed what's missing in large language models and his new joint embedding predictive architecture
- Theory of consciousness and potential for AI systems to exhibit features of consciousness
- Self-supervised learning revolutionized natural language processing
- Large language models lack a world model and generative models are difficult to represent uncertain predictions
- Successful in audio but not images, so need to predict a representation of the image


In [18]:
# Generating more accurate and context-aware summaries with 'refine'
# It generates the summary of the first chunk; 
# Then, for each successive chunk, the summary is integrated with new info from the new chunk.
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Craig Smith interviews Jan LeCoon, a deep learning developer and proponent of self-supervised
learning, about his new joint embedding predictive architecture and his theory of consciousness. Jan
discusses the gap in large language models and the potential for AI systems to exhibit features of
consciousness. He explains how self-supervised learning has revolutionized natural language
processing through the use of transformer architectures for pre-training, such as taking a piece of
text, removing some of the words, and replacing them with black markers to train a large neural net
to predict the words that are missing. This technique has been used in practical applications such
as contact moderation systems on Facebook, Google, YouTube, and more. Jan also explains how this
technique can be used to represent uncertain predictions in generative models, such as predicting
the missing words in a text, or predicting the missing frames in a video. He further explains that
while this techniqu

**Working with multiple video URLs. Adding Transcripts to DeepLake.**

In [19]:
# Loading video files from multiple URLs
import yt_dlp

def download_mp4_from_youtube(urls, job_id):
    # This will hold the titles and authors of each downloaded video
    video_info = []

    for i, url in enumerate(urls):
        # Set the options for the download
        file_temp = f'./data/{job_id}_{i}.mp4'
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
            'outtmpl': file_temp,
            'quiet': True,
        }

        # Download the video file
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            title = result.get('title', "")
            author = result.get('uploader', "")

        # Add the title and author to our list
        video_info.append((file_temp, title, author))

    return video_info


urls=["https://www.youtube.com/watch?v=mBjPyte2ZZo&t=78s",
    "https://www.youtube.com/watch?v=cjs7QKJNVYM",]

vides_details = download_mp4_from_youtube(urls, 1)

In [34]:
import whisper

# Load the model
model = whisper.load_model("base")

# iterate through each video and transcribe
results = []
for video in vides_details:
    print(f"Transcribing {video[0]}")
    result = model.transcribe(video[0])
    results.append( result['text'] )
    # print(f"Transcription for {video[0]}:\n{result['text']}\n")

Transcribing ./data/1_0.mp4


/Users/iryna/Documents/projects/langchain_snippets/langenv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribing ./data/1_1.mp4


/Users/iryna/Documents/projects/langchain_snippets/langenv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [27]:
with open ('output/mult_text.txt', 'w') as file:
    for r in results:
        file.write(r)

In [28]:
# Load the texts from the file and split the text to chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the texts
with open('output/mult_text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

# Split 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )
texts = text_splitter.split_text(text)

In [29]:
# Pack all the chunks into a Documents
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [30]:
# Build a DeepLake database with embedded documents
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

my_activeloop_org_id = "iryna"
my_activeloop_dataset_name = "youtube_summarizer"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_documents(docs)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://iryna/youtube_summarizer already exists, loading from the storage


|

Dataset(path='hub://iryna/youtube_summarizer', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['b14c8faa-3b0b-11ee-9ddc-12ee7aa5dbdc',
 'b14c9248-3b0b-11ee-9ddc-12ee7aa5dbdc',
 'b14c92d4-3b0b-11ee-9ddc-12ee7aa5dbdc',
 'b14c9338-3b0b-11ee-9ddc-12ee7aa5dbdc']

In [31]:
# Construct a retriever object
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4 #search for k the most relevant documents

In [32]:
# Constract prompt template with the QA chain
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of transcripts from a video to answer the question in bullet points and summarized. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Summarized answer in bullter points:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [33]:
from langchain.chains import RetrievalQA

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)


print(qa.run("Summarize the mentions of google according to their AI program"))



• Google uses self-supervised learning to train AI systems.
• This involves taking a piece of text, removing some of the words, and replacing them with black markers.
• The system then learns good representations of text that can be used for downstream tasks, such as translation or hitchbitch detection.
• This technique is used in contact moderation systems on Google, YouTube, and other applications.
• Large language models are partially based on this technique, which is used to predict the next word in a text.


<hr>
<a class="anchor" id="voice_assistant">
    
## 3. Creating Voice Assistant
    
</a>

<hr>
<a class="anchor" id="code_understanding">
    
## 4. Code Comprehension - Twitter Algorithm
    
</a>

<hr>
<a class="anchor" id="recommendation">
    
## 5. Recommendation Engine for Songs
    
</a>

<hr>
<a class="anchor" id="critique">
    
## 6. Self-Critique Chain
    
</a>

Self-critique chain acts as a mechanism to ensure model responses are appropriate in a production environment. By iterating over the model's output and checking against predefined expectations, the self-critique chain prompts the model to correct itself when necessary. 

<hr>
<a class="anchor" id="resources">
    
## 7. Additional Resources
    
</a>

- [Langchain documentation on Chains](https://python.langchain.com/docs/modules/chains/)
- [Textwrap Package](https://docs.python.org/3/library/textwrap.html)
- [Introducing Whisper](https://openai.com/research/whisper)
- [Deep Lake Vector Store in LangChain](https://docs.activeloop.ai/tutorials/vector-store/deep-lake-vector-store-in-langchain)
- [Voice Assitant - ‘JarvisBase’ repository on GitHub](https://github.com/peterw/JarvisBase)